In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import sklearn
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn import ensemble # for random forest



from sklearn.model_selection import GridSearchCV


%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')

In [8]:
# Import Cleaned Data 

data = pd.read_csv('../Ames_HousePrice_processed.csv')

In [9]:
data.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,909176150,856.0,126000.0,30,RL,0.0,7890.0,Pave,No Alley,Reg,...,166.0,0.0,No Pool,No Fence,No Misc Feature,0.0,3,2010,WD,Normal
1,905476230,1049.0,139500.0,120,RL,42.0,4235.0,Pave,No Alley,Reg,...,0.0,0.0,No Pool,No Fence,No Misc Feature,0.0,2,2009,WD,Normal
2,911128020,1001.0,124900.0,30,C (all),60.0,6060.0,Pave,No Alley,Reg,...,0.0,0.0,No Pool,No Fence,No Misc Feature,0.0,11,2007,WD,Normal
3,535377150,1039.0,114000.0,70,RL,80.0,8146.0,Pave,No Alley,Reg,...,111.0,0.0,No Pool,No Fence,No Misc Feature,0.0,5,2009,WD,Normal
4,534177230,1665.0,227000.0,60,RL,70.0,8400.0,Pave,No Alley,Reg,...,0.0,0.0,No Pool,No Fence,No Misc Feature,0.0,11,2009,WD,Normal


In [49]:
features = data.drop(['PID', 'SalePrice'], axis = 1)
target = data.iloc[:,2]

# To prepare features for one-hot encoding, make sure that all categorical features are dtype str
features = features.astype({'MSSubClass':'string','OverallQual':'string', 'OverallCond':'string'})

In [50]:
features['OverallQual'].name

'OverallQual'

In [94]:
# One hot encoding for regression analysis for all categorical variables
from pandas.api.types import is_numeric_dtype
def dummy_func(data):
    if len(data.shape) == 1:
        column = data
        largest_col = column.name + "_" + column.value_counts().index[0] # this will get the first of the sorted values
        column = pd.get_dummies(column, prefix = column.name, prefix_sep="_")
        column = column.drop(largest_col, axis = 1)
        return column
    return pd.concat([data[col] if is_numeric_dtype(data[col]) else dummy_func(data[col]) for col in data.columns], axis = 1)

# Apply dummy function
features = dummy_func(features)

features.head()

,GrLivArea,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_VWD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial
0,856.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1049.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1001.0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1039.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1665.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Train / test / split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

# RANDOM FOREST MODELS WITH HYPER-PARAMETER TUNING

In [53]:
# Instantiate model with 1000 decision trees
rf = ensemble.RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Fit simple default value model
# Train the model on training data
rf.fit(X_train, y_train);



In [54]:
# Evaluation
print('Random Forest model training R2 score: ', rf.score(X_train, y_train))
print('Random Forest model testing R2 score: ', rf.score(X_test, y_test))

# Accuracy
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error in Housing Price Prediction:', round(np.mean(errors), 2), '$USD')

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Random Forest model training R2 score:  0.9834151865190182
Random Forest model testing R2 score:  0.9013786446307004
Mean Absolute Error in Housing Price Prediction: 15811.15 $USD
Accuracy: 90.91 %.


In [59]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: GrLivArea            Importance: 0.24
Variable: GarageCars           Importance: 0.21
Variable: YearBuilt            Importance: 0.13
Variable: TotalBsmtSF          Importance: 0.06
Variable: 1stFlrSF             Importance: 0.05
Variable: BsmtQual_Ex          Importance: 0.04
Variable: GarageArea           Importance: 0.04
Variable: LotArea              Importance: 0.02
Variable: YearRemodAdd         Importance: 0.02
Variable: MasVnrArea           Importance: 0.02
Variable: BsmtFinSF1           Importance: 0.02
Variable: LotFrontage          Importance: 0.01
Variable: OverallQual_8        Importance: 0.01
Variable: ExterQual_Gd         Importance: 0.01
Variable: BsmtUnfSF            Importance: 0.01
Variable: CentralAir_N         Importance: 0.01
Variable: 2ndFlrSF             Importance: 0.01
Variable: FullBath             Importance: 0.01
Variable: KitchenQual_Ex       Importance: 0.01
Variable: Fireplaces           Importance: 0.01
Variable: OpenPorchSF          Importanc

In [65]:
# Top 5... 
feature_importance = list(zip(features.columns, rf.feature_importances_))
dtype = [('feature', 'S10'), ('importance', 'float')]
feature_importance = np.array(feature_importance, dtype=dtype)
feature_sort = np.sort(feature_importance, order='importance')[::-1]
[i for (i, j) in feature_sort[0:5]]

[b'GrLivArea', b'GarageCars', b'YearBuilt', b'TotalBsmtS', b'1stFlrSF']

## RANDOM FOREST HYPER-PARAMETER TUNING

-- random search is preferred for computational speed
-- we will use random search to narrow in on appropriate ranges 
-- then use grid search to fine tune within a smaller range

In [71]:
print('Parameters currently in use:\n')
print(rf.get_params())

from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 400, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt'] # auto is just n_features

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = True # I think we want to bootstrap 
oob_score = True




# set the parameter grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'oob_score': oob_score}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)


Parameters currently in use:

{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestRegressor(n_estimators=1000,
                                                   random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1200; total time=   2.6s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=   2.8s
[CV] END bootstrap=False, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=800; total time=   1.9s
[CV] END bootstrap=False, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=600; total time=   1.9s
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=  33.8s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=800; total time=   2.6s
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=   3.9s
[CV] END bootstrap=False, max_depth=3

In [88]:
def evaluate(model, test_features, test_labels, train_features, train_labels):
    r2_test = model.score(test_features, test_labels)
    r2_train = model.score(train_features, train_labels)
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} $USD.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('R-squared train = {:0.2f}%.'.format(r2_train))
    print('R-squared test = {:0.2f}%.'.format(r2_test))
    return (accuracy, r2_train, r2_test)


base_model = ensemble.RandomForestRegressor(n_estimators = 100, random_state = 42, oob_score = True)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test, X_train, y_train)[0]

# TOP 10 BASE MODEL FEATURE IMPORTANCE


# Get numerical feature importances
importances = list(base_model.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[0:10]]

Model Performance
Average Error: 16076.0480 $USD.
Accuracy = 90.75%.
R-squared train = 0.98%.
R-squared test = 0.90%.
Variable: GrLivArea            Importance: 0.22
Variable: GarageCars           Importance: 0.22
Variable: YearBuilt            Importance: 0.13
Variable: TotalBsmtSF          Importance: 0.05
Variable: 1stFlrSF             Importance: 0.05
Variable: BsmtQual_Ex          Importance: 0.04
Variable: GarageArea           Importance: 0.04
Variable: LotArea              Importance: 0.02
Variable: YearRemodAdd         Importance: 0.02
Variable: MasVnrArea           Importance: 0.02


[None, None, None, None, None, None, None, None, None, None]

In [96]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test, X_train, y_train)[0]
print('Best model params:', best_random.get_params())
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

# Get numerical feature importances
importances = list(best_random.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[0:50]]

Model Performance
Average Error: 15444.1436 $USD.
Accuracy = 91.13%.
R-squared train = 1.00%.
R-squared test = 0.90%.
Best model params: {'bootstrap': False, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 400, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Improvement of 0.41%.
Variable: GrLivArea            Importance: 0.08
Variable: YearBuilt            Importance: 0.06
Variable: GarageCars           Importance: 0.06
Variable: TotalBsmtSF          Importance: 0.05
Variable: 1stFlrSF             Importance: 0.05
Variable: GarageArea           Importance: 0.05
Variable: BsmtQual_Ex          Importance: 0.04
Variable: FullBath             Importance: 0.04
Variable: YearRemodAdd         Importance: 0.03
Variable: BsmtFinSF1          

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]